# RAG 



## Dados - Livros

download books: https://drive.google.com/drive/folders/1yhocUZ1cGfZiBg63hBlH8mXe2oHXYRuT?usp=sharing

Estrutura:
```
books-rag/
├── raw/                         # originais (NÃO edite)
│   ├── pdf/
│   └── epub/
├── working/                     # intermediários
│   ├── converted/               # md/txt por capítulo ou seção
│   ├── ocr/                     # PDFs escaneados corrigidos
│   └── manifests/               # CSV/JSON com inventário e status
├── corpus/                      # texto limpo + chunks com metadados
│   ├── jsonl/                   # um chunk por linha
│   └── parquet/                 # (opcional) rápido p/ análise
├── index/                       # índices de busca
│   ├── faiss/
│   └── bm25/
└── evaluation/                  # perguntas, gabaritos, logs

### Extract Book Data

In [13]:
!ls

Books-20251007T165548Z-1-001	  books_rag	     README.md
Books-20251007T165548Z-1-001.zip  experiments.ipynb  src


In [ ]:
!python src/extract_book_data.py \
  --root "../books_rag" \
  --pdf_dir  "../books_rag/raw/epub" \
  --epub_dir "../books_rag/raw/pdf"


Traceback (most recent call last):
  File "/home/pdi_4/anaconda3/envs/rag_book/lib/python3.11/pathlib.py", line 1116, in mkdir
    os.mkdir(self, mode)
FileNotFoundError: [Errno 2] No such file or directory: '/home/books_rag/corpus/jsonl'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pdi_4/anaconda3/envs/rag_book/lib/python3.11/pathlib.py", line 1116, in mkdir
    os.mkdir(self, mode)
FileNotFoundError: [Errno 2] No such file or directory: '/home/books_rag/corpus'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pdi_4/Documents/Documentos/rag/src/extract_book_data.py", line 304, in <module>
    main()
  File "/home/pdi_4/Documents/Documentos/rag/src/extract_book_data.py", line 267, in main
    ensure_writable_dir(out_dir)
  File "/home/pdi_4/Documents/Documentos/rag/src/extract_book_data.py", line 48, in ensure_writable_dir
    path.mkdir(parent

## Tipo de RAG